In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# ---------------------------------------
# 1. CONNECT TO MT5
# ---------------------------------------
if not mt5.initialize():
    raise RuntimeError("MT5 initialization failed")

SYMBOL = "XAUUSDc"
TIMEFRAME = mt5.TIMEFRAME_M1   # fetch M1 to build range bars
BARS = 5000                    # number of historical bars to fetch

rates = mt5.copy_rates_from_pos(SYMBOL, TIMEFRAME, 0, BARS)
mt5.shutdown()

df = pd.DataFrame(rates)
df['time'] = pd.to_datetime(df['time'], unit='s')

# ---------------------------------------
# 2. BUILD RANGE BARS (CUSTOM RANGE)
# ---------------------------------------
RANGE_SIZE = 1.0  # gold moves a lot, you can use 1.0 or 0.5

range_bars = []

open_price = df['open'][0]
high = open_price
low = open_price

for price in df['close']:
    high = max(high, price)
    low = min(low, price)

    if high - low >= RANGE_SIZE:
        close_price = price
        range_bars.append([open_price, high, low, close_price])

        # reset
        open_price = close_price
        high = close_price
        low = close_price

# Convert to DataFrame
rb = pd.DataFrame(range_bars, columns=['open','high','low','close'])
rb['index'] = range(1, len(rb)+1)

# ---------------------------------------
# 3. PLOT RANGE BARS USING PLOTLY
# ---------------------------------------
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=rb['index'],
    open=rb['open'],
    high=rb['high'],
    low=rb['low'],
    close=rb['close'],
    increasing_line=dict(width=2),
    decreasing_line=dict(width=2)
))

fig.update_layout(
    title=f'XAUUSDc Range Chart (Range = {RANGE_SIZE})',
    xaxis_title="Range Bar Number",
    yaxis_title="Price",
    template="plotly_dark",
    width=1000,
    height=600
)

fig.show()
